<img src= "https://frenzy86.s3.eu-west-2.amazonaws.com/python/safetensor.png" width=1000>

# Safetensors: a simple, safe and faster way to store and distribute tensors.

Safetensors is a new simple format for storing tensors safely (as opposed to pickle) and that is still fast (zero-copy). Safetensors is really fast

safetensors and ONNX serve different purposessafetensors is a simple, safe, and fast file format for storing and loading tensors. It is a secure alternative to Python’s pickle utility, which is not secure and may contain malicious code that can be executed.

In summary, safetensors is used for storing and loading tensors in a safe and fast way, while ONNX is used for sharing models between different deep learning frameworks. Same applies for other model sharing frameworks.

https://medium.com/@mandalsouvik/safetensors-a-simple-and-safe-way-to-store-and-distribute-tensors-d9ba1931ba04

In [ ]:
import torch
from safetensors.torch import save_file

tensors = {
            "embedding": torch.zeros((2, 2)),
            "attention": torch.zeros((2, 3))
            }
save_file(tensors, "model.safetensors")

In [ ]:
from safetensors import safe_open

tensors = {}
with safe_open("model.safetensors", framework="pt", device=0) as f:
    for k in f.keys():
        tensors[k] = f.get_tensor(k) # loads the full tensor given a key
print(tensors)

{'attention': tensor([[0., 0., 0.],
        [0., 0., 0.]], device='cuda:0'), 'embedding': tensor([[0., 0.],
        [0., 0.]], device='cuda:0')}


## Lazy loading

Lazy loading is the ability to load only some tensors, or part of tensors for a given file. This is possible with safetensors.

Lazy loading is really important in cases when we have a large file containing many key and value pairs. This can be a metadata cache for large dataset. If we can load the value for single keys indivisually it will be memory efficient and faster else we will have to load the full file into memory to inspect any of the key.

In [ ]:
#do not run
# from safetensors.torch import load_model, save_model

# save_model(model, "model1.safetensors")
# # Instead of save_file(model.state_dict(), "model.safetensors")

# load_model(model, "model1.safetensors")
# # Instead of model.load_state_dict(load_file("model.safetensors"))

##PyTorch Operations
Load a model in PyTorch.



In [ ]:
from torchvision.models import resnet18

model_pt = resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 178MB/s]


Save the state_dict to safetensor, and load them back to a new model.



In [ ]:
from safetensors.torch import load_model, save_model

# save the state dict
save_model(model_pt, "resnet18.safetensors")

# load the model without weights
model_st = resnet18(pretrained=False)
load_model(model_st, "resnet18.safetensors")

(set(), [])

## HuggingFace to ONNX

<img src= "https://frenzy86.s3.eu-west-2.amazonaws.com/python/huggingonnx.png" width=1000>


reference to this article:
https://www.philschmid.de/optimizing-transformers-with-optimum

In [ ]:
!pip install transformers[onnx] torch -q
!pip install onnx -q
!pip install accelerate -U -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 993.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.2 which is incompatible.
tensorflow 2.1

In [ ]:
import warnings
warnings.filterwarnings('ignore')

transformers.onnx enables you to convert model checkpoints to an ONNX graph by leveraging configuration objects. That way you don’t have to provide the complex configuration for dynamic_axes etc.

## From PyTorch--(complicated)

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# load model and tokenizer
model_id = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)
dummy_model_input = tokenizer("This is a sample", return_tensors="pt")

# export
torch.onnx.export(
                    model,
                    tuple(dummy_model_input.values()),
                    f="torch-model.onnx",
                    input_names=['input_ids', 'attention_mask'],
                    output_names=['logits'],
                    dynamic_axes={'input_ids': {0: 'batch_size', 1: 'sequence'},
                                'attention_mask': {0: 'batch_size', 1: 'sequence'},
                                'logits': {0: 'batch_size', 1: 'sequence'}},
                    do_constant_folding=True,
                    opset_version=14, #chekc it if return the error of ONNXversion
                    )

## Exporting our checkpoint with the transformers.onnx (all in one with tokenizer)

In [ ]:
from pathlib import Path
import transformers
from transformers.onnx import FeaturesManager
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification

# load model and tokenizer
model_id = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# load config
model_kind, model_onnx_config = FeaturesManager.check_supported_model_or_raise(model)
onnx_config = model_onnx_config(model.config)

# export
transformers.onnx.export(
                        preprocessor=tokenizer,
                        model=model,
                        config=onnx_config,
                        opset=14,
                        output=Path("modelwithtokenizer1.onnx")
                        )

(['input_ids', 'attention_mask'], ['last_hidden_state'])

## mini inference

In [ ]:
import onnxruntime
import numpy as np

onnx_model_path = "modelwithtokenizer1.onnx"
session = onnxruntime.InferenceSession(onnx_model_path)

text = "I hate you."

# Tokenize input text
inputs = tokenizer(text, return_tensors="pt")
# Get model inputs
inputs_onnx = {k: v.numpy() for k, v in inputs.items()}
outputs_onnx = session.run(None, inputs_onnx)
print(np.argmax(outputs_onnx[0]))

0


# Export with Optimum (transformer pipeline)
Optimum Inference includes methods to convert vanilla Transformers models to ONNX using the ORTModelForXxx classes. To convert your Transformers model to ONNX you simply have to pass from_transformers=True to the from_pretrained() method and your model will be loaded and converted to ONNX leveraging the transformers.onnx package under the hood.

In [ ]:
!pip install optimum[onnxruntime] -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires 

The best part about the conversion with Optimum is that you can immediately use the model to run predictions or load it inside a pipeline.

In [ ]:
from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import pipeline

model_id = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = ORTModelForSequenceClassification.from_pretrained(
                                                           model_id, # onnx checkpoint
                                                           export=True
                                                           )

# save onnx checkpoint and tokenizer
onnx_path = Path("test")
model.save_pretrained(onnx_path)
tokenizer.save_pretrained(onnx_path)

('test/tokenizer_config.json',
 'test/special_tokens_map.json',
 'test/vocab.txt',
 'test/added_tokens.json',
 'test/tokenizer.json')

In [ ]:
from transformers import pipeline

vanilla_clf = pipeline("text-classification", model=model, tokenizer=tokenizer)
vanilla_clf("Could you assist me in finding my lost card?")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'NEGATIVE', 'score': 0.9986515641212463}]